In [2]:
from collections import Counter
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import gensim
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter, defaultdict
%matplotlib inline
from sklearn.model_selection import train_test_split

import sys

In [10]:
t = pd.read_excel(data_path)

In [12]:
t.head()

,label,text,title
0,14,convent intern commiss rhine bern convent le g...,environmental_protection
1,14,convent intern commiss rhine bern convent le g...,pollution_control_measures
2,14,convent intern commiss rhine bern convent le g...,environmental_cooperation
3,14,convent intern commiss rhine bern convent le g...,rhine_valley
4,14,convent intern commiss rhine bern convent le g...,ec_agreement


In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(t['title'])
t['target'] = le.transform(t['title'])

In [6]:
data_path = '/mnt/dump/df.xlsx'

train_path = '/mnt/dump/df_train.xlsx'
test_path = '/mnt/dump/df_test.xlsx'

In [14]:
t.head()

,label,text,title,target
0,14,convent intern commiss rhine bern convent le g...,environmental_protection,1278
1,14,convent intern commiss rhine bern convent le g...,pollution_control_measures,2806
2,14,convent intern commiss rhine bern convent le g...,environmental_cooperation,1271
3,14,convent intern commiss rhine bern convent le g...,rhine_valley,3157
4,14,convent intern commiss rhine bern convent le g...,ec_agreement,1072


In [20]:
def add_label(s):
    return '__' + 'label' + '__' + str(s)

In [15]:
train_data = pd.read_excel(train_path)

In [16]:
train_data['target'] = le.transform(train_data['title'])

In [18]:
test_data = pd.read_excel(test_path)

In [19]:
test_data['target'] = le.transform(test_data['title'])

In [22]:
train_data['label_new'] = train_data.target.apply(add_label)

In [23]:
test_data['label_new'] = test_data.target.apply(add_label)

In [30]:
len(test_data['label_new'].unique())

2007

In [27]:
train_data[['label_new', 'text']].to_csv('/mnt/dump/fasttext/train.txt', sep='\t',header=None, index=None)
test_data[['label_new', 'text']].to_csv('/mnt/dump/fasttext/test.txt', sep='\t',header=None, index=None)

In [63]:
e = pd.read_csv('~/fastText/test.txt',header = None,sep="\t")[1]

In [64]:
e.head()

0     decis eea joint committe amend annex ii techn...
1     decis eea joint committe juli amend annex xi ...
2     council regul ec amend regul ec common organi...
3     commiss regul ec decemb open provid administr...
4     commiss regul ec march amend regul ec special...
Name: 1, dtype: object

In [3]:
d = pd.read_excel('/mnt/dump/df_train.xlsx')

In [4]:
d.head()

,label,text,title
0,14,convent intern commiss rhine bern convent le g...,environmental_protection
1,14,convent intern commiss rhine bern convent le g...,pollution_control_measures
2,14,convent intern commiss rhine bern convent le g...,environmental_cooperation
4,14,convent intern commiss rhine bern convent le g...,ec_agreement
6,17,exchang letter commiss intern bureau weight me...,weights_and_measures


## title = y

In [102]:
w = pd.read_csv('~/fastText/probs.txt',header = None,sep="\t")

In [103]:
v = pd.read_csv('~/fastText/probs_train.txt',header = None,sep="\t")

In [104]:
from tqdm import tqdm
classes_ = [str(i) for i in tqdm(t['title'].unique())]

def to_standart_format(lst):
    res_matrix = np.zeros((len(lst), len(classes_)))
    for c, s in enumerate(lst):
        for line in s.split('__label__')[1:]:
            temp = line.split(' ')
            title, p = int(temp[0]), temp[1]
            p = float(p)
            res_matrix[c, title] = p
    return res_matrix


100%|██████████| 3956/3956 [00:00<00:00, 1351414.45it/s]


In [105]:
p = to_standart_format(w[0].get_values())

In [106]:
e = to_standart_format(v[0].get_values())

In [91]:
# d = {'y_train': train_data['label_new'],
#      'y_train_label': train_data['label'],
#      'p_train': e,
#      'y_test': test_data['label_new'],
#      'y_test_label': test_data['label'],
#       'p_test': p,
#        'classes_': classes_ }

In [107]:
d = {'p_train': e,
      'p_test': p,
     'classes': classes_
}

In [83]:
from tqdm import tqdm
classes = classes_
def foo(lst):
    if hasattr(lst, 'get_values'):
        lst = lst.get_values()
    res = np.zeros(len(classes))
    for i in lst:
        res[classes.index(i)] = 1
    return np.array(res)

y_truth_train = {k: foo(obj['title']) for k, obj in tqdm(train_data.groupby('label'))}

100%|██████████| 17377/17377 [00:06<00:00, 2662.43it/s]


In [86]:
y_truth_test = {k: foo(obj['title']) for k, obj in tqdm(test_data.groupby('label'))}

100%|██████████| 1931/1931 [00:00<00:00, 2609.87it/s]


In [92]:
label_index_train = {v: k for k, v in enumerate(train_data['label'])}
label_index_test = {v: k for k, v in enumerate(test_data['label'])}

In [94]:
from sklearn.metrics import roc_auc_score, average_precision_score

roc_list_train, ap_list_train = [], []
for label_id, lst in tqdm(y_truth_train.items()):
    y_ = lst
    ind = label_index_train[label_id]
    p_ = e[ind, :]
    roc_list_train.append(roc_auc_score(y_, p_))
    ap_list_train.append(average_precision_score(y_, p_))

In [96]:
print('ROC AUC : ', np.mean(roc_list_train))
print('RAP      : ', np.mean(ap_list_train))

ROC AUC :  0.89683945180137634
RAP     :  0.29005936254193846


In [100]:
from sklearn.metrics import roc_auc_score, average_precision_score

roc_list_test, ap_list_test = [], []
for label_id, lst in tqdm(y_truth_test.items()):
    y_ = lst
    ind = label_index_test[label_id]
    p_ = p[ind, :]
    roc_list_test.append(roc_auc_score(y_, p_))
    ap_list_test.append(average_precision_score(y_, p_))

100%|██████████| 1931/1931 [00:04<00:00, 397.81it/s]


In [101]:
print('ROC AUC : ', np.mean(roc_list_test))
print('AP     : ', np.mean(ap_list_test))
# print("""ROC AUC :  0.89683945180137634
# RAP     :  0.29005936254193846""")
# ROC AUC :  0.5000033553940071
# RAP     :  0.0026659013707067995

ROC AUC :  0.5000033553940071
RAP     :  0.0026659013707067995


In [108]:
import pickle
pickle.dump(d, open('/mnt/dump/dirty_dump/fasttext|fast_text|1', 'wb'))

In [73]:
train_data

,label,text,title,target,label_new
0,14,convent intern commiss rhine bern convent le g...,environmental_protection,1278,__label__1278
1,14,convent intern commiss rhine bern convent le g...,pollution_control_measures,2806,__label__2806
2,14,convent intern commiss rhine bern convent le g...,environmental_cooperation,1271,__label__1271
4,14,convent intern commiss rhine bern convent le g...,ec_agreement,1072,__label__1072
6,17,exchang letter commiss intern bureau weight me...,weights_and_measures,3883,__label__3883
7,17,exchang letter commiss intern bureau weight me...,technical_cooperation,3567,__label__3567
8,17,exchang letter commiss intern bureau weight me...,scientific_cooperation,3232,__label__3232
9,38,council decis juli conclus protocol access uni...,gatt,1688,__label__1688
10,38,council decis juli conclus protocol access uni...,tariff_preference,3532,__label__3532
11,38,council decis juli conclus protocol access uni...,protocol_to_an_agreement,2957,__label__2957
